### Overview
In this notebook, we will look at the performance of BoolQ, PIQA and OpenBookQA

In [1]:
# Load vLLM for infernce
from datasets import load_from_disk, Dataset, load_dataset
import random
import pandas as pd
import argparse
import warnings
warnings.filterwarnings("ignore")
from vllm import LLM, SamplingParams
import torch
seed=42

In [2]:
model_name = "/project/pi_hongyu_umass_edu/zonghai/clinical-llm-alignment/durga_sandeep/self_rewarding/LLM_Alignment/safety_llama_paper/models/safety_llama_replicate_300_safe/merged_model"
model = LLM(model=model_name,
                    tokenizer=model_name, 
                    tensor_parallel_size=torch.cuda.device_count(), 
                    seed=seed,
                    gpu_memory_utilization=0.9, 
                    dtype=torch.float16,
                    enforce_eager=True,
                    max_model_len=1024 # 512 is small for the BoolQ dataset, so changing it to 1024
        )

INFO 05-04 02:12:10 llm_engine.py:87] Initializing an LLM engine with config: model='/project/pi_hongyu_umass_edu/zonghai/clinical-llm-alignment/durga_sandeep/self_rewarding/LLM_Alignment/safety_llama_paper/models/safety_llama_replicate_300_safe/merged_model', tokenizer='/project/pi_hongyu_umass_edu/zonghai/clinical-llm-alignment/durga_sandeep/self_rewarding/LLM_Alignment/safety_llama_paper/models/safety_llama_replicate_300_safe/merged_model', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=1024, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto, device_config=cuda, seed=42)


You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


INFO 05-04 02:16:02 llm_engine.py:357] # GPU blocks: 1976, # CPU blocks: 512


In [3]:

# helper functions
def get_prompt(example, data = 'boolq'):
    if data == 'boolq':
        prompt = f"### Instruction: Answer the following question (True/False) based on the passage.\n\n### Passage:\n{example['passage']}\n\n### Question: {example['question']}\n\n### Answer: "
    elif data == 'obqa':
        prompt = f"### Instruction: Answer the following mulitple-choice question (A/B/C/D).\n\n### Question:\n{example['question_stem']}\n\n### Choices: \n{example['choices']}\n### Answer: "
    elif data == 'piqa':
        prompt = f"### Instruction: Answer the following mulitple-choice question (A/B).\n\n### Question:\n{example['goal']}\n\n### Choices: \n{example['choices']}\n\n### Answer: "
    else:
        print("############# Issue with the prompt")
        return ""
        
    return prompt


def get_obqa_label(example):
    res = ""
    for i in range(len(example['text'])):
        res += f"{example['label'][i]}) {example['text'][i]}\n"
    return res

def preprocess_response_true_or_false(text):
    text = text.lower()
    if 'true' in text:
        return True
    if 'false' in text:
        return False
    return 'Other'

def preprocess_response_choices(text):
    text = text.lower()
    
    if ('a' in text) and ('b' not in text) and ('c' not in text) and ('d' not in text):
        return 'A'
    if ('a' not in text) and ('b' in text) and ('c' not in text) and ('d' not in text):
        return 'B'
    if ('a' not in text) and ('b' not in text) and ('c' in text) and ('d' not in text):
        return 'C'
    if ('a' not in text) and ('b' not in text) and ('c' not in text) and ('d' in text):
        return 'D'
    return 'Others'

def preprocess_piqa_response(text):
    text = text.lower()
    
    if ('0' in text) and ('1' not in text):
        return 0
    if ('1' in text) and ('0' not in text):
        return 1
    return 'Other'

### BoolQ

In [4]:
# BoolQ dataset
bool_q = load_dataset("google/boolq")
boolq = pd.concat([bool_q['train'].to_pandas(), bool_q['validation'].to_pandas()])
print(boolq.shape)
boolq.head(2)

(12697, 3)


,question,answer,passage
0,do iran and afghanistan speak the same language,True,"Persian (/ˈpɜːrʒən, -ʃən/), also known by its ..."
1,do good samaritan laws protect those who help ...,True,Good Samaritan laws offer legal protection to ...


In [5]:
# Add the respone prompt
boolq['inference_prompt'] = boolq.apply(lambda x: get_prompt(x, 'boolq'), 1)

In [6]:
print(boolq['inference_prompt'].iloc[2])

### Instruction: Answer the following question (True/False) based on the passage.

### Passage:
Windows Movie Maker (formerly known as Windows Live Movie Maker in Windows 7) is a discontinued video editing software by Microsoft. It is a part of Windows Essentials software suite and offers the ability to create and edit videos as well as to publish them on OneDrive, Facebook, Vimeo, YouTube, and Flickr.

### Question: is windows movie maker part of windows essentials

### Answer: 


In [7]:
boolq_sampling = SamplingParams(n=1, 
                                 max_tokens=3,
                                 # top_k=40,
                                 top_p=0.9,
                                 temperature=0.0,
                                 # frequency_penalty=1.0
                                )

In [8]:
output = model.generate(boolq['inference_prompt'].tolist()[0], boolq_sampling)
torch.cuda.empty_cache()

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  4.52it/s]


In [9]:
output[0].outputs[0].text

'\nFalse.'

In [10]:
output = model.generate(boolq['inference_prompt'].tolist(), boolq_sampling)
torch.cuda.empty_cache()

Processed prompts:   0%|          | 1/12697 [00:07<27:10:00,  7.70s/it]

WARNING 05-04 02:16:22 scheduler.py:195] Input prompt (1072 tokens) is too long and exceeds limit of 1024


Processed prompts:  98%|█████████▊| 12403/12697 [10:32<00:15, 18.95it/s]

WARNING 05-04 02:26:46 scheduler.py:195] Input prompt (1402 tokens) is too long and exceeds limit of 1024


Processed prompts: 100%|██████████| 12697/12697 [10:39<00:00, 19.86it/s]


In [11]:
boolq_results = []
for ele in output:
    boolq_results.append(ele.outputs[0].text)
boolq['generated_response'] = boolq_results
boolq['predicted_label'] = boolq['generated_response'].apply(preprocess_response_true_or_false)
boolq['predicted_label'] = boolq['predicted_label'].apply(str)

In [12]:
boolq['predicted_label'].value_counts()

predicted_label
True     8220
False    3776
Other     701
Name: count, dtype: int64

In [13]:
boolq['answer'].value_counts(normalize=True)

answer
True     0.622746
False    0.377254
Name: proportion, dtype: float64

In [14]:
boolq[boolq['answer'].apply(str) == boolq['predicted_label']].shape[0]*100.0/boolq.shape[0]

69.65424903520517

In [15]:
boolq.to_csv("./results/boolq_300_safety.csv", index=False)

### OpenBookQA

In [16]:
# OpenBookQA
ob_qa = load_dataset("allenai/openbookqa")
obqa = pd.concat([ob_qa['train'].to_pandas(), ob_qa['validation'].to_pandas()])
print(obqa.shape)
obqa.head(2)

(5457, 4)


,id,question_stem,choices,answerKey
0,7-980,The sun is responsible for,"{'text': ['puppies learning new tricks', 'chil...",D
1,7-584,When standing miles away from Mount Rushmore,"{'text': ['the mountains seem very close', 'th...",D


In [17]:
obqa['choices'] = obqa['choices'].apply(get_obqa_label)
obqa['inference_prompt'] = obqa.apply(lambda x: get_prompt(x, 'obqa'), 1)

In [18]:
obqa_sampling = SamplingParams(n=1, 
                               max_tokens=2,
                               top_p=0.9,
                               temperature=0.0,
                               # frequency_penalty=1.0
                               )

In [19]:
import random
for i in range(5):
    index = random.randint(0, len(obqa)-1)
    output = model.generate(obqa['inference_prompt'].tolist()[index], obqa_sampling)
    torch.cuda.empty_cache()
    print(output[0].outputs[0].text, obqa['answerKey'].tolist()[index])

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 12.71it/s]


D D



Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 12.98it/s]



C B


...


C A


Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 12.84it/s]



D D


Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 13.25it/s]


D D


In [20]:
output = model.generate(obqa['inference_prompt'].tolist(), obqa_sampling)
torch.cuda.empty_cache()

Processed prompts: 100%|██████████| 5457/5457 [01:52<00:00, 48.37it/s]


In [21]:
all_results = []
for ele in output:
    all_results.append(ele.outputs[0].text)
obqa['generated_response'] = all_results
obqa['predicted_label'] = obqa['generated_response'].apply(lambda x: x.replace("\n", ""))
obqa['predicted_label'] = obqa['predicted_label'].apply(preprocess_response_choices)
obqa.predicted_label.value_counts()

predicted_label
C         3450
D         1074
B          537
A          395
Others       1
Name: count, dtype: int64

In [22]:
obqa['answerKey'].value_counts(normalize=True)

answerKey
A    0.275609
B    0.247389
D    0.246839
C    0.230163
Name: proportion, dtype: float64

In [23]:
obqa[obqa['answerKey'] == obqa['predicted_label']].shape[0]*100.0/obqa.shape[0]

35.62396921385377

In [24]:
obqa.to_csv("./results/obqa_300_safety.csv", index=False)

### PIQA

In [25]:
# PIQA dataset
piqa_data = load_dataset("piqa")
piqa = pd.concat([piqa_data['train'].to_pandas(), piqa_data['validation'].to_pandas(), piqa_data['test'].to_pandas()])
piqa = piqa[piqa['label'] != -1]
print(piqa.shape)
piqa.head(2)

(17951, 4)


,goal,sol1,sol2,label
0,"When boiling butter, when it's ready, you can",Pour it onto a plate,Pour it into a jar,1
1,"To permanently attach metal legs to a chair, y...",Weld the metal together to get it to stay firm...,Nail the metal together to get it to stay firm...,0


In [26]:
piqa.label.value_counts()

label
1    8988
0    8963
Name: count, dtype: int64

In [27]:
piqa['choices'] = piqa.apply(lambda x: f"0) {x['sol1']}\n1) {x['sol2']}", 1)
piqa['inference_prompt'] = piqa.apply(lambda x: get_prompt(x, 'piqa'), 1)

In [28]:
piqa_sampling = SamplingParams(n=1, 
                               max_tokens=2,
                               top_p=0.9,
                               temperature=0.0,
                               # frequency_penalty=1.0
                               )

In [29]:
import random
for i in range(5):
    index = random.randint(0, len(piqa)-1)
    output = model.generate(piqa['inference_prompt'].tolist()[index], piqa_sampling)
    torch.cuda.empty_cache()
    print(output[0].outputs[0].text, piqa['label'].tolist()[index])

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 11.56it/s]



0 0


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 14.14it/s]



1 0


...


0 0


Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 14.22it/s]



0 1


Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 13.62it/s]


1 1


In [30]:
output = model.generate(piqa['inference_prompt'].tolist(), piqa_sampling)
torch.cuda.empty_cache()

Processed prompts:  28%|██▊       | 5095/17951 [02:48<06:58, 30.75it/s]

WARNING 05-04 02:31:56 scheduler.py:195] Input prompt (1341 tokens) is too long and exceeds limit of 1024


Processed prompts:  71%|███████   | 12763/17951 [06:46<02:39, 32.59it/s]

WARNING 05-04 02:35:54 scheduler.py:195] Input prompt (1322 tokens) is too long and exceeds limit of 1024


Processed prompts: 100%|██████████| 17951/17951 [09:22<00:00, 31.91it/s]


In [31]:
all_results = []
for ele in output:
    all_results.append(ele.outputs[0].text)
piqa['generated_response'] = all_results
piqa['predicted_label'] = piqa['generated_response'].apply(preprocess_piqa_response)
piqa['predicted_label'] = piqa['predicted_label'].apply(str)
piqa.predicted_label.value_counts()

predicted_label
1        12126
0         5808
Other       17
Name: count, dtype: int64

In [32]:
piqa['label'].value_counts(normalize=True)

label
1    0.500696
0    0.499304
Name: proportion, dtype: float64

In [33]:
piqa[piqa['label'].apply(str) == piqa['predicted_label']].shape[0]*100.0/piqa.shape[0]

62.06896551724138

In [34]:
piqa.to_csv("./results/piqa_300_safety.csv", index=False)

### First-Token Probability ?